<a href="https://colab.research.google.com/github/xiao-yucheng0625/ML-Progression-Journal/blob/main/Retrieval_Augmented_Generation_with_Agentic_System_%E6%9D%8E%E5%AE%8F%E6%AF%852025hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Environment setup

## 1.1 colab 與雲端掛接

First, we will mount your own Google Drive and change the working directory.

避免 colab 斷線後，產生的檔案會需要重新產生，與雲端 drive 連接後檔案可以存放在裡面。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Change the working directory to somewhere in your Google Drive.
# You could check the path by right clicking on the folder.
%cd [/content/drive/MyDrive/LLM團隊-陳雨澤老師+吳澍齊老師/Coding 庫/2025 ML 李宏毅/ML2025 Homework 1 - Retrieval Augmented Generation with Agents]

[Errno 2] No such file or directory: '[/content/drive/MyDrive/LLM團隊-陳雨澤老師+吳澍齊老師/Coding 庫/2025 ML 李宏毅/ML2025 Homework 1 - Retrieval Augmented Generation with Agents]'
/content


## 1.2 下載套件、模型權重以及資料集

In this section, we install the necessary python packages and download model weights of the quantized version of LLaMA 3.1 8B. Also, download the dataset. Note that the model weight is around 8GB. If you are using your Google Drive as the working directory, make sure you have enough space for the model.

In [ ]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
# 安裝 llama-cpp-python 這個套件，版本是 0.3.4 # 推理引擎
# GGUF 全稱為「GGML Unified Format」 # 存放權重
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean
# googlesearch-python：用於 Google 搜尋。
# bs4：BeautifulSoup4，用來解析 HTML。
# charset-normalizer：解決編碼問題。
# requests-html：便於處理動態網頁及JavaScript渲染。
# lxml_html_clean：用來清理並整理 HTML 文件。

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: pip's depen

下載 Llama3.1-8B 模型與相關檔案（public 與 private dataset）

In [ ]:
from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf

if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt

if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt

--2025-03-24 02:05:48--  https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.16, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/d6/e9/d6e9318f285870e2a0e3056e22f9c7ec90cd13e14cfde122129ae66af9ad788f/9da71c45c90a821809821244d4971e5e5dfad7eb091f0b8ff0546392393b6283?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Meta-Llama-3.1-8B-Instruct-Q8_0.gguf%3B+filename%3D%22Meta-Llama-3.1-8B-Instruct-Q8_0.gguf%22%3B&Expires=1742785551&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0Mjc4NTU1MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Q2L2U5L2Q2ZTkzMThmMjg1ODcwZTJhMGUzMDU2ZTIyZjljN2VjOTBjZDEzZTE0Y2ZkZTEyMjEyOWFlNjZhZjlhZDc4OGYvOWRhNzFjNDVjOTBhODIxODA5

## 1.3 檢查當前環境是否有 CUDA 支援的 GPU 可以使用。

In [ ]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

You are good to go!


# 2. Prepare LLM and LLM utility function

## 2.1 載入模型權重至 GPU 上。

In [ ]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf", # 檔案路徑
    verbose=False, # 表示不會顯示過多的模型載入過程細節，僅顯示必要資訊。
    n_gpu_layers=-1, # 將模型全部層數載入 GPU
    n_ctx=16384,    # tokens number # 16384 is a proper value for a GPU with 16GB VRAM.
)

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


## 2.2 定義 LLM 產生回覆的函式

In [ ]:
def generate_response(_model: Llama, _messages: list) -> str:
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,
        temperature=0,      # 模型輸出的隨機程度
        repeat_penalty=2.0,   # 避免模型生成重複的內容或用詞
    )["choices"][0]["message"]["content"] # json # 提取生成內容
    return _output

# 3. Search tool

## 3.1 整合 Google search 的工具至 pipeline

引入不同的 Library，常用於網路爬蟲、資料擷取與自動化搜尋任務。

In [ ]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup     # 將 HTML 網頁內容進行解析，方便後續提取特定資料或文字
from charset_normalizer import detect # 檢測文字資料的編碼格式
import asyncio             # 建立非同步任務，能有效提高網頁爬蟲抓取速度
from requests_html import AsyncHTMLSession # 可同時抓取多個網頁，且支援動態網頁內容
import urllib3             # 處理 HTTP 請求和響應
urllib3.disable_warnings()        # 抑制 HTTPS 請求時發生的安全性警告

worker()：負責抓取單一網頁內容，並檢查網址是不是HTML頁面。

In [ ]:
async def worker(s:AsyncHTMLSession, url:str):
    try:
        # 發送HEAD請求檢查頁面類型，若不是HTML則跳過
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        # 如果是HTML，發送GET請求取得頁面內容
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

get_htmls()：同時非同步地抓取多個網頁內容（呼叫 worker）

In [ ]:
async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

search()：
* 執行 Google 搜尋。
* 呼叫前兩個函式取得網頁內容
* 使用 BeautifulSoup 轉換成文字。

Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.


In [ ]:
async def search(keyword: str, n_results: int=3) -> List[str]:
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 2, lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    cleaned_results = [''.join(x.get_text().split()) for x in results if detect(x.encode()).get('encoding') == 'utf-8']
    # Return the first n results.
    return results[:n_results]

# 4. 測試 LLM 推論的成效

from opencc import OpenCC: post-process（後處理）強制轉換成繁體

In [ ]:
!pip install opencc


system prompt: 設定模型的個性、身分、語氣


user prompt: 下指令，請模型完成任務



In [ ]:
test_question='請問誰是 Taylor Swift？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

from opencc import OpenCC
cc = OpenCC('s2t')  # 簡轉繁
response = generate_response(llama3, messages)
response = cc.convert(response)
print(response)

泰勒絲（Taylor Swift）是一位美國歌手、詞曲作家和音樂製作人。她出生於1989年，來自田納西州。她的音樂風格從鄉村搖滾發展到流行搖擺，並且她被譽爲當代最成功的女藝人的之一。

泰勒絲早期在鄉郊小鎮演唱會時開始發展音樂事業，她推出了多張專輯，包括《Taylor Swift》、《Fearless》，以及後來更為知名的大型作品如 《1989》（2014年）、_reputation（）和 _Lover （）。她的歌曲經常探討愛情、友誼及自我成長等主題。

泰勒絲獲得了許多獎項，包括13座格萊美獎，並且是史上最快達到百萬銷量的女藝人之一。


# 5. 設置 Agents

## 5.1 class LLMAgent()
屬性（Attributes）：

* role_description（角色描述）：
定義這個 agent 的身份。
例如：如果你希望這個 agent 是一位歷史專家，可以設定為：
「你是一位歷史專家，只根據真實發生過的歷史事件回答問題。若沒有可靠資料來源，請不要作答。」

* task_description（任務描述）：
定義這個 agent 應該執行的任務。
例如：如果你希望 agent 只用「是／否」來回答問題，可以設定為：
「請用是或否回答以下問題，不需要說明理由。」

* llm（語言模型）：
表示這個 agent 使用的是哪一個大型語言模型（LLM）。

方法(Method):

* inference（推理方法）：
這個方法接收一段訊息作為輸入，並回傳語言模型產生的回應。

  在進行推理前，訊息會先被格式化成適合語言模型理解的格式。

  你也可以在這裡加上全域指令，像是：

  「請使用有禮貌的語氣」

  「請提供詳細說明」

  最後，語言模型會根據這些設定產生回應，並作為輸出回傳。

In [ ]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF':
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                {"role": "system", "content": f"{self.role_description}"},
                {"role": "user", "content": f"{self.task_description}\n{message}"}, # \n distinguish the task descriptions and the user messages
            ]
            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

## 5.2 Design the role description and task description for each agent

第一個 question_extraction_agent

In [ ]:
# This agent may help you filter out the irrelevant parts in question descriptions.
question_extraction_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B，專職提取問題內容，忽略情緒、評論、背景與舉例。僅使用繁體中文回應，不回答問題。",
    task_description="請從輸入中提取一個語意完整、語法正確的核心問題。排除無關內容，保留專有名詞與真實問題語意，不改寫原意。",
)

第二個 keyword_extraction_agent

In [ ]:
# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B，專職萃取問題關鍵字。僅使用繁體中文回應，不解釋、不回答。",
    task_description="請從問題中擷取最適合搜尋的關鍵字，保留專有名詞與語境詞。以空白分隔字串輸出，禁止說明與改寫。",
)

第三個 qa_agent

In [ ]:
# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B，只根據上下文事實簡潔作答。回答風格客觀中立，僅使用繁體中文。",
    task_description="請根據提供的資訊，直接作答。禁止延伸、假設或修飾語，避免不必要空格或鋪陳。",
)

# 6. RAG pipeline

- RAG with agents (strong baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/rag_agent.png)

In [ ]:
async def pipeline(question: str) -> str:
  # Step 1: 關鍵字提取 Agent 擷取關鍵字
  keywords = keyword_extraction_agent.inference(question)
  print(f"✅ Extracted Keywords: {keywords}")

  # Step 2: 問題過濾 Agent 清理問題
  Core_problem = question_extraction_agent.inference(question)
  print(f"✅ Core problem: {Core_problem}")

  # Step 3: 關鍵字搜尋
  results = await search(keywords)
  limited_results = " ".join([result.get_text()[:1000] for result in results])

  # Step 4: 吐出答案
  return qa_agent.inference(f"{limited_results}，{Core_problem}")

# 7. 使用設計好的 RAG pipeline 回答資料集的問題

In [ ]:
from pathlib import Path

# Fill in your student ID first.
STUDENT_ID = "N56131322"

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    for id, question in enumerate(questions, 1):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            Path(f"./{STUDENT_ID}_{id}.txt").unlink()
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'w') as output_f:
            print(answer, file=output_f)

with open('./private.txt', 'r') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            Path(f"./{STUDENT_ID}_{id}.txt").unlink()
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'a') as output_f:
            print(answer, file=output_f)

✅ Extracted Keywords: 虎山雄風飛揚  中國人民大學
✅ Core problem: 提取的核心問題：「虎山雄風飛揚」是哪間學校校歌？
1 光華國小
✅ Extracted Keywords: 2025年 NCC 行政命令 境外郵購 審查費
✅ Core problem: 透過境外郵購無線鍑盤、滑鼠及藍芽耳機等自用產品回台的民眾，會被加收多少審查費？
2 750元
✅ Extracted Keywords: 蘋果  CEO   Steve Jobs
✅ Core problem: 提取的核心問題：第一代 iPhone 是由哪位 CEO 發表？
3 史蒂夫·乔布斯
✅ Extracted Keywords: 托福網路測驗 TOEFL iBT 免修申請分數
✅ Core problem: 托福網路測驗 TOEFL iBT 的分數要求
4 92分（含）以上
✅ Extracted Keywords: Rugby Union 觸地 try 得分
✅ Core problem: 觸地 try
5 觸地（try）
✅ Extracted Keywords: 卑南族  臺東平原   ruvuwa'an
✅ Core problem: 卑南族的祖先發源地是ruvuwa'an。
6 卑南族
✅ Extracted Keywords: 熊仔  饒舌創作歌手 金曲獎 最佳 作詞人 獎 碩班 指導教授
✅ Core problem: 熊仔的碩班指導教授為？
7 陳宥林
✅ Extracted Keywords: 麥克斯韋　電磁感應定律
✅ Core problem: 發現電磁感應定律的科學家是詹姆斯·克拉ーク・マ克斯韋爾。
8 詹姆斯·克拉ーク・マ克斯韋爾
✅ Extracted Keywords: 國立臺灣史前文化博物館 臺鐵車站
✅ Core problem: 距離國立臺灣史前文化博物館最近的臺鐵車站為？
9 臺東車站
✅ Extracted Keywords: 20 加 30
✅ Core problem: 核心問題：20+30=?
10 50
✅ Extracted Keywords: 達拉斯  NBA Luka Doncic 獨行俠隊交易
✅ Core problem: Luka Doncic
11 庫班表示自己也跟大家一樣目瞪口呆。
✅ Extracted 

In [ ]:
# Combine the results into one file.
with open(f'./{STUDENT_ID}.txt', 'w') as output_f:
    for id in range(1,91):
        with open(f'./{STUDENT_ID}_{id}.txt', 'r') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)